# GFS data

NOAA's Global Forecasting System (GFS) is a service that offers historical weather forecasts and re-analysis data (i.e., forecasts that are more accurate because they use historical information, but are still not 100% reliable).

Their data can be found on their FTP server ftp://nomads.ncdc.noaa.gov/GFS/analysis_only/. Your username must be "anonymous" but your password can be anything, though they ask that it be your email address. To get the data:
1. Log onto the FTP server
2. Go to a directory YYYYMM
3. In that directory, go to a subdirectory YYYYMMDD
4. Here you will find files in 3 formats: grb, grb2, and inv. inv are just index files so I ignore those. grb2 are GRIB files in the new format, grb are GRIB files in the old format. Our collaborators use the grb2 (which are only available 2007 onwards), so that's what I use.
5. You will have several grb2 options of the form gfsanl_4_YYYYMMDD_HHMM_OOO.grb2 where HHMM is the 24-hour format UTC time of the weather, and OOO is the hour offset. So a file ending in 0600_003.grb will be weather data collected at 0600 UTC and interpolated to 3 + 0600 = 0900 UTC. These offets are important for getting cumulative data (like rain): the \_000 files just have instantaneous data, but the \_006 files have 6 hours of (approximate) cumulative data. I use 000 files for instantaneous weather variables (temp, humidity, wind) and 4 006 files for daily rain.
6. Process the data. This data is by far the most annoying to deal with. Python has a library called pygrib for dealing with GRIB data. These files have hundreds of layers, with a value at each half-degree lat/on for each layer, so they get fairly large. My solution to that has been to download one file, get the 4 layers I care about for Alaska only, and discard to save space.

The layers I've been using are as follows:
* Temperature: layer name "Temperature", type of level "surface"
* Humidity: name is either "Surface air relative humidity", "2 metre relative humidity", or "Relative humidity" (level 2) depending on the version of pygrib.
* Wind: wind is stored in vector form, with a U-component layer and a V-component layer. I just take the magnitude of this (U,V) vector and store it. You want layers named "10 metre U wind component" and "10 metre V wind component".
* Rain. Again, you need 4 different files to get 24 hours of rain. I use 3AM local time on the day until 3AM local time the next day, which is pretty ad hoc. The layer name is "Total Precipitation" and again, only exists in \_003 and \_006 files.

In [ ]:
import os
REP_DIR = "/home/cagraff/Documents/dev/fire_prediction/"
os.chdir(REP_DIR)
from data import data

In [ ]:
gfs = data.load_gfs_dict()
gfs.keys()

I've stored this data as a dictionary of lists/tensors. The tensors have 3 dimensions, lat, lon, and day:
* days: a list of tuples representing the day of the third dimension of each tensor. Note that some days are missing in the data so these will not always be consecutive. (These days are missing on the public servers but NOAA has this data somewhere! One of the back-burner goals of the project is to pester the NOAA people to give us all this data)
* lats: a 2d matrix representing the latitude at each 2d slice of a tensor
* lons: a 2d matrix representing the longitude at each 2d slice of a tensor. For example, if the temp tensor has a value 273 at  position (x,y,z), that means the temperature of day days[z] at latitude lats[x,y] and longitude lons[x,y] is 273 Kelvin.
* temp: Temperature tensor (in Kelvin)
* humidity: Humidity tensor (in percent)
* vpd: Vapor Pressure Defecit tensor (in percent/100). This is a derivated measure from temp and humidity representing how dry the air is relative to how dry it could be given the temp.
* wind: Wind magnitude tensor (in m/s)
* rain: Rain sum tensor (in mm/day)
* valid_bits: A list of 4-tuples representing how many of the 4 tensors for rain I successfully pulled from the server. So if valid_bits[z] = (1,1,0,1), that means that the sums in rain[x,y,z] incorporate data from tensors 1, 2, and 4, but not 3.

The data is kind of annoying to use in this format, but I've created some functions that make it easier to use for specific points/regions:
* geometry.grid_conversion.gfs_to_loc_df(gfs_dict, lat, lon, outfi=None): take this dict gfs_dict and a specific latitude and longitude and convert it to a pandas DataFrame
* geometry.grid_conversion.get_gfs_for_region(day, month, year, gfs_dict, bb=ak_inland_bb): just get a subset of this gfs_dict that only has values for the day (day,month,year) in the region of the boundind box bb

In [ ]:
# Get a pandas DataFrame for Fairbanks and plot some weather covariates over the course of 2013
from geometry.grid_conversion import gfs_to_loc_df, fairbanks_lat_lon
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

lat,lon = fairbanks_lat_lon
gfs_df = gfs_to_loc_df(gfs, lat, lon)
gfs_df = gfs_df[(gfs_df.year==2013)]

covars = ["temp", "humidity", "wind", "rain"]
#ticks = [[294, 298, 302, 306, 310], [20, 30, 40, 50], [0,2,4,6], [0., .5, 1., 1.5, 2., 2.5]]
#ylims = [(293, 310), (20, 55), (), ()]
titles = ["Temperature (K)", "Humidity (%)", "Wind (m/s)", "Rain (mm/day)"]
fig, axes = plt.subplots(nrows=len(covars), ncols=1, figsize=(10,8))
for i, var in enumerate(covars):
    matplotlib.rcParams.update({'font.size': 15})
    if not i:
        ax = plt.subplot(411 + i)
        ax1 = ax
    else:
        ax = plt.subplot(411 + i, sharex=ax1)

    plt.plot(gfs_df.dayofyear, gfs_df[var], "r-", linewidth=3)
        
    if i != len(covars) - 1:
        plt.tick_params(axis='x', labelbottom='off') 
    else:
        plt.xlabel("Day of year (2013)")
#    plt.yticks(ticks[i])
#    if ylims[i]:
#        plt.ylim(ylims[i])
    matplotlib.rcParams.update({'font.size': 13})
    plt.title(titles[i], fontweight='bold')
fig.tight_layout(h_pad=.1)